In [1]:
import pandas as pd
import numpy as np
import datetime


In [2]:
dane = pd.read_csv('Data/df_enriched.csv')
pd.options.display.max_columns=100
dane.head()

,Unnamed: 0,data_zdarzenia,godzina,GPS_x,GPS_y,miejscowosc,ulica,numer_domu,numer_drogi,pikietaz,obszar,skrzyzowanie,geometria_drogi,charakterystyka_miejsca,oswietlenie,rodzaj_zdarzenia,wina,rodzaj_drogi,nawierzchnia_twarda,stan_nawierzchni,sygnalizacja,oznakowanie_poziome,liczba_pasow,zachowanie_kierujacego,zachowanie_pieszego,inne_przyczyny,liczba_rannych,liczba_zabitych,liczba_kolizji,liczba_wypadkow,rok,miesiac,dzien_miesiąca,dzien_tygodnia,data,czy_weekend,handlowa,Skrzyżowanie,warunki_atmosferyczne,pogoda_dobre_warunki,pogoda_pochmurno,pogoda_mgla,pogoda_dym,pogoda_oslep_slonce,pogoda_opady,pogoda_snieg,pogoda_grad,pogoda_deszcz,pogoda_wiatr,nawierzchnia_sucha,nawierzchnia_mokra,nawierzchnia_oblodzona,nawierzchnia_zasniezona,nawierzchnia_nierowna,nawierzchnia_kaluze,nawierzchnia_zanieszyszczenia,zachowanie_kierujacego_niepr_cofanie,zachowanie_kierujacego_niepr_omijanie,zachowanie_kierujacego_niedost_predkosci,zachowanie_kierujacego_nieudzielenie_pierw,zachowanie_kierujacego_niepr_skrecanie,zachowanie_kierujacego_niezachowanie_odl,zachowanie_kierujacego_niepr_zmiana_pasa,zachowanie_kierujacego_zmeczenie,zachowanie_kierujacego_niepr_wymijanie,zachowanie_kierujacego_nieprzestrz_znakow,zachowanie_kierujacego_niestos_do_sygn,zachowanie_kierujacego_nieust_przejazdu,zachowanie_kierujacego_nieust_pieszemu,zachowanie_kierujacego_gwalt_hamowanie,zachowanie_kierujacego_niepr_wyprzedzanie,zachowanie_kierujacego_niepr_zatrzymanie,zachowanie_kierujacego_niepr_zawracanie,zachowanie_kierujacego_niepr_przejazd_rowerowy,zachowanie_kierujacego_jazda_bez_swiatel,zachowanie_kierujacego_inne_przyczyny
0,0,2016-01-01,5,18.635806,50.369722,GLIWICE,TOSZECKA,NaN,901,61.6,0.0,NaN,odcinek prosty,jezdnia,Noc - droga nieoświetlona,Najechanie na zwierzę,z innych przyczyn,jednojezdniowa dwukierunkowa,NaN,sucha,0,0.0,2.0,NaN,NaN,"Obiekty, zwierzęta na drodze",0,0,1,0,2016,1,1,4,2016-01-01,0,2,NaN,dobre warunki atmosferyczne,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,2016-01-01,0,19.180833,50.323083,DĄBROWA GÓRNICZA,SIENKIEWICZA,NaN,G200 030S,NaN,1.0,Z drogą z pierwsz.,NaN,jezdnia,Noc - droga oświetlona,Najechanie na pojazd unieruchomiony,z winy kierującego,jednojezdniowa dwukierunkowa,NaN,sucha,0,0.0,NaN,nieprawidłowe cofanie,NaN,NaN,0,0,1,0,2016,1,1,4,2016-01-01,0,2,2.0,dobre warunki atmosferyczne,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,2016-01-01,18,18.877639,50.445083,TARNOWSKIE GÓRY,KS.IGNACEGO SIWCA,4A,GMINNA,NaN,1.0,NaN,zakręt łuk,jezdnia,Noc - droga oświetlona,Najechanie na pojazd unieruchomiony,z winy kierującego,jednojezdniowa dwukierunkowa,NaN,sucha,0,1.0,2.0,nieprawidłowe omijanie,NaN,NaN,0,0,1,0,2016,1,1,4,2016-01-01,0,2,NaN,dobre warunki atmosferyczne,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,2016-01-01,15,18.860583,50.444667,TARNOWSKIE GÓRY,JÓZEFA PIŁSUDSKIEGO,6,P3276S,NaN,1.0,NaN,odcinek prosty,jezdnia,Dzień,Zderzenie pojazdów tylne,z winy kierującego,jednojezdniowa dwukierunkowa,NaN,sucha,0,0.0,2.0,nieprawidłowe omijanie,NaN,NaN,0,0,1,0,2016,1,1,4,2016-01-01,0,2,NaN,pochmurno,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,2016-01-01,17,19.047556,50.466806,OŻAROWICE,DWORCOWA,16,P3203S,NaN,1.0,NaN,odcinek prosty,jezdnia,"Świt, zmrok",Najechanie na zwierzę,z innych przyczyn,jednojezdniowa dwukierunkowa,NaN,sucha,0,1.0,2.0,NaN,NaN,"Obiekty, zwierzęta na drodze",0,0,1,0,2016,1,1,4,2016-01-01,0,2,NaN,dobre warunki atmosferyczne,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [3]:
#przekształcam nazwy miejscowosci aby ułatwić wysyłanie zapytań na stronę www.worldweatheronline.com


def simplify(a):  
    a = a.lower()
    a = a.replace(' ','_')
    a = a.replace('ą','a')                   
    a = a.replace('ę','e')                  
    a = a.replace('ć','c')                    
    a = a.replace('ł','l')                    
    a = a.replace('ń','n')                   
    a = a.replace('ó','o')                  
    a = a.replace('ś','s')
    a = a.replace('ź','z')
    a = a.replace('ż','z')
          
    return(a)

def simplify_column(serie):
    a = serie.unique()                       
    a = np.array2string(a)                  
    return(simplify(a))

def simplify_cell(string):
    return(simplify(str(string)))

In [5]:
dane['miejscowosc']=dane['miejscowosc'].apply(lambda x: simplify_cell(x))

In [6]:
location_list = dane['miejscowosc']

In [7]:
location_list = location_list.unique()

<span style=color:blue> Pobieramy API 

In [7]:
#pip install wwo-hist

In [8]:
# instaluję pakiet

from wwo_hist import retrieve_hist_data

In [9]:
import os     #tworzę folder, w którym będę przechowywać dane o pogodzie dla podanej miejscowości

os.chdir("C:/Users/HP/DataWorkshop/Data/WEATHER_HOURLY")

In [10]:
import os 

def change_name(path):
    remove_list = os.listdir(path) #otrzymuję listę miast, dla których pogodę już podbrałam
    new_list = []
    for i in remove_list:
        if i == '.ipynb_checkpoints':   #usuwam niepotrzebne pliki
            remove_list = remove_list.remove(i)
        else:
            i = i.replace('.csv','')  #usuwam 'csv' z nazwy pliku, a następnie nazwy miast dodaję do mojej nowej listy
            new_list.append(i)
        
    return new_list

def removing(location_list, new_list):
    
    for j in location_list:
        if j in new_list:
            location_list = list(location_list)  #w location_list pozostają nazwy miast, dla których pogoda nie została jeszcze pobrana
            location_list.remove(j)
    return location_list

new_list = change_name('C:/Users/HP/DataWorkshop/Data/WEATHER_HOURLY')

location_list = removing(location_list, new_list)

print((location_list))

['trachy', 'kieleczka', 'twardorzeczka', 'konieczki', 'wiklow', 'gorzelnia', 'rzeniszow', 'panoszow', 'topolow', 'przybedza', 'szalsza', 'niegowka', 'sobuczyna', 'swietoszowice', 'lutrowskie', 'koszwice', 'krasawa', 'borowiany', 'godziszka', 'drogobycza', 'warezyn', 'szymocice', 'hanusek', 'siemierzyce', 'laryszow', 'lekartow', 'pustkowie_lgockie', 'skowronow', 'trzepizury', 'starokrzepice', 'cyganka', 'wilgoszcza', 'laczka', 'kowale', 'zaborze', 'odrzykon', 'skarzyce', 'grzegorzowice', 'wisnicze', 'laka', 'napoleon', 'zrebice', 'widzowek', 'sokolniki', 'stanica', 'studzienice', 'zawada_ksiazeca', 'kusieta', 'wola_kiedrzynska', 'twardowice', 'potepa', 'walaszczyki', 'targoszyce', 'ostreznik', 'gajowice', 'kopienica', 'florkow', 'rudyszwald', 'lelity', 'soblowka', 'lubowice', 'srubarnia', 'latosowka', 'zbrojewsko', 'wedzina', 'gaszczyk', 'oczko', 'antonow', 'wieszczeta', 'leboszowice', 'karsow', 'pniaki_mokrzeskie', 'korwinow', 'grodczanki', 'chrobakowe', 'koclin', 'kozakow', 'pluzniczk

In [11]:
frequency =  6          #częstotliwość pobierania inforamcji 1-24
start_date = '01-JAN-2016'     #data rozpoczęcia pobierania danych
end_date = '31-DEC-2019'       #data zakończenia pobierania danych
api_key = ''       #klucz poboeramy po zarejestrowaniu się na stronie www.worldweatheronline.com/developer
location_list = location_list

for i in location_list:
    try:
        hist_weather_data = retrieve_hist_data(api_key,
                                    location_list,
                                    start_date,
                                    end_date,
                                    frequency,
                                    location_label = False,
                                    export_csv = True,
                                    store_df = True)
    
    except KeyError as e:
        continue
    




Retrieving weather data for trachy


Currently retrieving data for trachy: from 2016-01-01 to 2016-01-31


HTTPError: HTTP Error 401: Unauthorized

In [12]:
print(len(os.listdir('C:/Users/HP/DataWorkshop/Data/WEATHER_HOURLY')))

1016


In [14]:
'''new_location_list=[]
for i in location_list:
    if i in dane['miejscowosc'].unique():
        new_location_list.append((dane['GPS_x'][i],dane['GPS_y'][i]))
    else:
        pass
print(new_location_list[0][0])
codes, uniques = pd.factorize(['b', 'b', 'a', 'c', 'b','a','a','b','b','c','c','c','c'], sort=True)
print(codes)

print(uniques)'''

"new_location_list=[]\nfor i in location_list:\n    if i in dane['miejscowosc'].unique():\n        new_location_list.append((dane['GPS_x'][i],dane['GPS_y'][i]))\n    else:\n        pass\nprint(new_location_list[0][0])\ncodes, uniques = pd.factorize(['b', 'b', 'a', 'c', 'b','a','a','b','b','c','c','c','c'], sort=True)\nprint(codes)\n\nprint(uniques)"

maxtempC - maksymalna temperatura w ciągu dnia w stopniach C

mintempC - minimalna temperatura w ciągu dnia w stopniach C

totalSnow_cm - całkowita ilość opadu śniegu w cm

sunHour	- ilolść godzin słonecznych 

uvIndex	 - indeks UV

moon_illumination - (%)

moonrise - 'wschód' księżyca (datetime)

moonset - 'zachód' księżyca (datetime)

sunrise - wschód słońca (datetime)

sunset - zachód słońca (datetime)

DewPointC - Temperatura punktu rosy w stopniach C

FeelsLikeC - temperatura odczuwalna w stopniach C 

HeatIndexC - Temperatura indeksu cieplnego w stopniach C
WindChillC - #odczuwalna temperatura w stopniach C
WindGustKmph - podmuchy wiatru w km na godzinę
cloudcover - zachmurzenie (%)
humidity - wilgotność (%)	
precipMM - opady w milimetrach	
pressure - Ciśnienie atmosferyczne w milibarach (mb)
tempC - temperatura w stopniach Celsjusza 
visibility - widoczność w km
winddirDegree - kierunek wiatru w stopniach
windspeedKmph  - prędkość wiatru w km na godzinę
location - lokalizacja

In [15]:
files = os.listdir('C:/Users/HP/DataWorkshop/Data/WEATHER_HOURLY')

result = pd.concat([pd.read_csv(f) for f in files], ignore_index=True) #łączę pliki csv zawierające dane pogodowe dla danej miejscowościnw jeden plik csv


In [16]:
result.columns

Index(['date_time', 'maxtempC', 'mintempC', 'totalSnow_cm', 'sunHour',
       'uvIndex', 'moon_illumination', 'moonrise', 'moonset', 'sunrise',
       'sunset', 'DewPointC', 'FeelsLikeC', 'HeatIndexC', 'WindChillC',
       'WindGustKmph', 'cloudcover', 'humidity', 'precipMM', 'pressure',
       'tempC', 'visibility', 'winddirDegree', 'windspeedKmph', 'location'],
      dtype='object')

In [17]:
#zmieniam typ kolumny z datą

result['date_time'] = pd.to_datetime(result['date_time'])

#result.maxtempC.unique()
#temp_bins = [-29, -20, -10,0, 10, 20, 30, 40, 50]
#pd.cut(result['maxtempC'],bins=temp_bins)
#result.maxtempC=pd.cut(result['maxtempC'],bins=temp_bins)
result.groupby(['location']).describe()

maxtempC                                              \
                        count       mean       std   min  25%   50%   75%   
location                                                                    
adamow                 5844.0  13.175907  9.968555 -15.0  5.0  14.0  22.0   
adamowice              5844.0  13.970568  9.436266 -12.0  6.0  14.0  22.0   
albertow               5844.0  13.657769  9.477353 -10.0  6.0  14.0  22.0   
aleksandria            5844.0  13.830938  9.340507  -9.0  6.0  14.0  22.0   
aleksandria_pierwsza   5844.0  13.452430  9.585914 -12.0  6.0  14.0  22.0   
...                       ...        ...       ...   ...  ...   ...   ...   
zwierzyniec_pierwszy   5844.0  11.982204  9.822297 -16.0  3.0  12.0  21.0   
zwierzyniec_trzeci     5844.0  12.356605  9.866860 -15.0  4.0  12.0  21.0   
zwonowice              5844.0  13.970568  9.436266 -12.0  6.0  14.0  22.0   
zytna                  5844.0  13.970568  9.436266 -12.0  6.0  14.0  22.0   
zywiec                 5844.0  11.978097  9.343587 -17.0  4.0  12.0  20.0   

                           mintempC                                            \
                       max    count      mean       std   min  25%  50%   75%   
location                                                                        
adamow                33.0   5844.0  5.988364  7.425219 -17.0  0.0  6.0  12.0   
adamowice             33.0   5844.0  6.868583  7.197788 -16.0  1.0  7.0  13.0   
albertow              34.0   5844.0  6.730322  7.132676 -15.0  1.0  7.0  13.0   
aleksandria           34.0   5844.0  6.760438  7.010427 -13.0  1.0  7.0  13.0   
aleksandria_pierwsza  34.0   5844.0  6.673511  7.198383 -16.0  1.0  7.0  13.0   
...                    ...      ...       ...       ...   ...  ...  ...   ...   
zwierzyniec_pierwszy  33.0   5844.0  4.902122  7.332825 -20.0  0.0  5.0  11.0   
zwierzyniec_trzeci    32.0   5844.0  5.279945  7.259428 -18.0  0.0  5.0  11.0   
zwonowice             33.0   5844.0  6.868583  7.197788 -16.0  1.0  7.0  13.0   
zytna                 33.0   5844.0  6.868583  7.197788 -16.0  1.0  7.0  13.0   
zywiec                31.0   5844.0  4.841205  7.365807 -22.0 -1.0  5.0  11.0   

                           totalSnow_cm                                     \
                       max        count      mean       std  min  25%  50%   
location                                                                     
adamow                21.0       5844.0  0.123340  0.555731  0.0  0.0  0.0   
adamowice             22.0       5844.0  0.128542  0.656421  0.0  0.0  0.0   
albertow              22.0       5844.0  0.049692  0.405129  0.0  0.0  0.0   
aleksandria           22.0       5844.0  0.076249  0.409360  0.0  0.0  0.0   
aleksandria_pierwsza  22.0       5844.0  0.129021  0.647374  0.0  0.0  0.0   
...                    ...          ...       ...       ...  ...  ...  ...   
zwierzyniec_pierwszy  20.0       5844.0  0.126010  0.519753  0.0  0.0  0.0   
zwierzyniec_trzeci    20.0       5844.0  0.146270  0.586570  0.0  0.0  0.0   
zwonowice             22.0       5844.0  0.128542  0.656421  0.0  0.0  0.0   
zytna                 22.0       5844.0  0.128542  0.656421  0.0  0.0  0.0   
zywiec                21.0       5844.0  0.279124  1.269997  0.0  0.0  0.0   

                                sunHour                                     \
                      75%   max   count      mean       std  min  25%  50%   
location                                                                     
adamow                0.0  10.1  5844.0  9.279877  4.019730  3.0  5.5  8.9   
adamowice             0.0  11.1  5844.0  9.675428  3.832972  3.0  6.8  9.9   
albertow              0.0  11.3  5844.0  9.638877  4.075590  3.0  6.3  9.9   
aleksandria           0.0   8.2  5844.0  9.766872  4.187210  3.0  6.4  9.9   
aleksandria_pierwsza  0.0  13.9  5844.0  9.542300  3.964180  3.0  6.1  9.8   
...                   ...   ...     ...       ...       ...  ...  ...  ...   
zwierzyniec_

In [29]:
result['mintempC'].min()
result[result['mintempC']==-35]

,date_time,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,moonrise,moonset,sunrise,sunset,DewPointC,FeelsLikeC,HeatIndexC,WindChillC,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph,location
3078476,2019-02-07 00:00:00,-28,-35,0.0,5.2,2,14,09:29 AM,07:54 PM,08:15 AM,05:35 PM,-34,-39,-33,-39,10,14,89,0.0,1016,-33,10,254,5,mochala
3078477,2019-02-07 06:00:00,-28,-35,0.0,5.2,2,14,09:29 AM,07:54 PM,08:15 AM,05:35 PM,-36,-42,-35,-42,13,11,89,0.0,1014,-35,10,247,6,mochala
3078478,2019-02-07 12:00:00,-28,-35,0.0,5.2,2,14,09:29 AM,07:54 PM,08:15 AM,05:35 PM,-31,-38,-29,-38,13,68,83,0.0,1014,-29,10,260,10,mochala
3078479,2019-02-07 18:00:00,-28,-35,0.0,5.2,2,14,09:29 AM,07:54 PM,08:15 AM,05:35 PM,-32,-40,-31,-40,18,100,87,0.0,1013,-31,10,288,10,mochala
4727780,2019-12-28 00:00:00,-28,-35,0.0,2.9,2,8,02:49 PM,08:06 PM,01:00 PM,04:58 PM,-28,-36,-27,-36,26,10,90,0.0,1015,-27,10,339,12,sol
4727781,2019-12-28 06:00:00,-28,-35,0.0,2.9,2,8,02:49 PM,08:06 PM,01:00 PM,04:58 PM,-30,-39,-29,-39,26,4,92,0.0,1013,-29,10,335,12,sol
4727782,2019-12-28 12:00:00,-28,-35,0.0,2.9,2,8,02:49 PM,08:06 PM,01:00 PM,04:58 PM,-29,-39,-29,-39,27,12,94,0.0,1012,-29,5,335,13,sol
4727783,2019-12-28 18:00:00,-28,-35,0.0,2.9,2,8,02:49 PM,08:06 PM,01:00 PM,04:58 PM,-29,-34,-28,-34,26,1,91,0.0,1011,-31,10,122,13,sol


In [18]:
#wybieram interesujące mnie columny

result = result[['date_time', 'maxtempC', 'mintempC','tempC','totalSnow_cm','cloudcover', 'humidity', 'visibility','precipMM','pressure','moon_illumination','location']]
result.dtypes

date_time            datetime64[ns]
maxtempC                      int64
mintempC                      int64
tempC                         int64
totalSnow_cm                float64
cloudcover                    int64
humidity                      int64
visibility                    int64
precipMM                    float64
pressure                      int64
moon_illumination             int64
location                     object
dtype: object

In [19]:
result['hour'] = pd.DatetimeIndex(result['date_time']).hour
result

,date_time,maxtempC,mintempC,tempC,totalSnow_cm,cloudcover,humidity,visibility,precipMM,pressure,moon_illumination,location,hour
0,2016-01-01 00:00:00,-6,-8,-6,0.0,0,40,10,0.0,1032,50,adamow,0
1,2016-01-01 06:00:00,-6,-8,-7,0.0,0,46,10,0.0,1031,50,adamow,6
2,2016-01-01 12:00:00,-6,-8,-6,0.0,28,58,10,0.0,1031,50,adamow,12
3,2016-01-01 18:00:00,-6,-8,-7,0.0,73,55,10,0.0,1032,50,adamow,18
4,2016-01-02 00:00:00,-8,-11,-8,0.0,34,56,10,0.0,1032,43,adamow,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5937499,2019-12-30 18:00:00,3,-6,-1,0.0,44,70,10,0.0,1032,22,zywiec,18
5937500,2019-12-31 00:00:00,1,-9,-1,0.0,39,69,10,0.0,1029,26,zywiec,0
5937501,2019-12-31 06:00:00,1,-9,-1,0.0,2,68,10,0.0,1026,26,zywiec,6
5937502,2019-12-31 12:00:00,1,-9,1,0.0,85,70,10,0.0,1026,26,zywiec,12


In [20]:
dane.dtypes

Unnamed: 0                                          int64
data_zdarzenia                                     object
godzina                                             int64
GPS_x                                             float64
GPS_y                                             float64
                                                   ...   
zachowanie_kierujacego_niepr_zatrzymanie            int64
zachowanie_kierujacego_niepr_zawracanie             int64
zachowanie_kierujacego_niepr_przejazd_rowerowy      int64
zachowanie_kierujacego_jazda_bez_swiatel            int64
zachowanie_kierujacego_inne_przyczyny               int64
Length: 76, dtype: object

In [21]:
dane['data_zdarzenia'] = pd.to_datetime(dane['data_zdarzenia'])

In [22]:
def change_hour(df):
    for i in df['godzina'][:5]:
        print(df['godzina'])

In [23]:
#dane zostały pobrane z częstotliwością 6 godzin; grupuję godziny wypadków na 4 grupy: 0-6, 7-12, 13-18, 19-24
def change_hour(df):
    if (0 <df['godzina'] < 6):
        df['godzina'] = 0
    elif (6 < df['godzina'] < 12):
        df['godzina'] = 6
    elif (12 < df['godzina'] < 18):
        df['godzina'] = 12
    elif (18 < df['godzina'] < 24):
        df['godzina'] = 18
    return df['godzina']          

dane['godzina'] = dane.apply(change_hour, axis=1)

In [24]:
dane['godzina'].unique()

array([ 0, 18, 12,  6], dtype=int64)

In [25]:
merge_data = pd.merge(dane,result , left_on=['data_zdarzenia','miejscowosc','godzina'], right_on =['date_time','location','hour'], how ='left')
merge_data.head(10)

,Unnamed: 0,data_zdarzenia,godzina,GPS_x,GPS_y,miejscowosc,ulica,numer_domu,numer_drogi,pikietaz,obszar,skrzyzowanie,geometria_drogi,charakterystyka_miejsca,oswietlenie,rodzaj_zdarzenia,wina,rodzaj_drogi,nawierzchnia_twarda,stan_nawierzchni,sygnalizacja,oznakowanie_poziome,liczba_pasow,zachowanie_kierujacego,zachowanie_pieszego,inne_przyczyny,liczba_rannych,liczba_zabitych,liczba_kolizji,liczba_wypadkow,rok,miesiac,dzien_miesiąca,dzien_tygodnia,data,czy_weekend,handlowa,Skrzyżowanie,warunki_atmosferyczne,pogoda_dobre_warunki,pogoda_pochmurno,pogoda_mgla,pogoda_dym,pogoda_oslep_slonce,pogoda_opady,pogoda_snieg,pogoda_grad,pogoda_deszcz,pogoda_wiatr,nawierzchnia_sucha,nawierzchnia_mokra,nawierzchnia_oblodzona,nawierzchnia_zasniezona,nawierzchnia_nierowna,nawierzchnia_kaluze,nawierzchnia_zanieszyszczenia,zachowanie_kierujacego_niepr_cofanie,zachowanie_kierujacego_niepr_omijanie,zachowanie_kierujacego_niedost_predkosci,zachowanie_kierujacego_nieudzielenie_pierw,zachowanie_kierujacego_niepr_skrecanie,zachowanie_kierujacego_niezachowanie_odl,zachowanie_kierujacego_niepr_zmiana_pasa,zachowanie_kierujacego_zmeczenie,zachowanie_kierujacego_niepr_wymijanie,zachowanie_kierujacego_nieprzestrz_znakow,zachowanie_kierujacego_niestos_do_sygn,zachowanie_kierujacego_nieust_przejazdu,zachowanie_kierujacego_nieust_pieszemu,zachowanie_kierujacego_gwalt_hamowanie,zachowanie_kierujacego_niepr_wyprzedzanie,zachowanie_kierujacego_niepr_zatrzymanie,zachowanie_kierujacego_niepr_zawracanie,zachowanie_kierujacego_niepr_przejazd_rowerowy,zachowanie_kierujacego_jazda_bez_swiatel,zachowanie_kierujacego_inne_przyczyny,date_time,maxtempC,mintempC,tempC,totalSnow_cm,cloudcover,humidity,visibility,precipMM,pressure,moon_illumination,location,hour
0,0,2016-01-01,0,18.635806,50.369722,gliwice,TOSZECKA,NaN,901,61.6,0.0,NaN,odcinek prosty,jezdnia,Noc - droga nieoświetlona,Najechanie na zwierzę,z innych przyczyn,jednojezdniowa dwukierunkowa,NaN,sucha,0,0.0,2.0,NaN,NaN,"Obiekty, zwierzęta na drodze",0,0,1,0,2016,1,1,4,2016-01-01,0,2,NaN,dobre warunki atmosferyczne,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-01-01,-2.0,-5.0,-4.0,0.0,8.0,48.0,10.0,0.0,1030.0,50.0,gliwice,0.0
1,1,2016-01-01,0,19.180833,50.323083,dabrowa_gornicza,SIENKIEWICZA,NaN,G200 030S,NaN,1.0,Z drogą z pierwsz.,NaN,jezdnia,Noc - droga oświetlona,Najechanie na pojazd unieruchomiony,z winy kierującego,jednojezdniowa dwukierunkowa,NaN,sucha,0,0.0,NaN,nieprawidłowe cofanie,NaN,NaN,0,0,1,0,2016,1,1,4,2016-01-01,0,2,2.0,dobre warunki atmosferyczne,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-01-01,-2.0,-6.0,-5.0,0.0,0.0,43.0,10.0,0.0,1030.0,50.0,dabrowa_gornicza,0.0
2,2,2016-01-01,18,18.877639,50.445083,tarnowskie_gory,KS.IGNACEGO SIWCA,4A,GMINNA,NaN,1.0,NaN,zakręt łuk,jezdnia,Noc - droga oświetlona,Najechanie na pojazd unieruchomiony,z winy kierującego,jednojezdniowa dwukierunkowa,NaN,sucha,0,1.0,2.0,nieprawidłowe omijanie,NaN,NaN,0,0,1,0,2016,1,1,4,2016-01-01,0,2,NaN,dobre warunki atmosferyczne,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2016-01-01,12,18.860583,50.444667,tarnowskie_gory,JÓZEFA PIŁSUDSKIEGO,6,P3276S,NaN,1.0,NaN,odcinek prosty,jezdnia,Dzień,Zderzenie pojazdów tylne,z winy kierującego,jednojezdniowa dwukierunkowa,NaN,sucha,0,0.0,2.0,nieprawidłowe omijanie,NaN,NaN,0,0,1,0,2016,1,1,4,2016-01-01,0,2,NaN,pochmurno,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2016-01-01,12,19.047556,50.466806,ozarowice,DWORCOWA,16,P3203S,NaN,1.0,NaN,odcinek prosty,jezdnia,"Świt, zmrok",Najechanie na zwierzę,z innych przyczyn,jednojezdniowa dwukierunkowa,NaN,sucha,0,1.0,2.0,NaN,NaN,"Obiekty, zwierzęta na drodze",0,0,1,0,2016,1,1,4,2016-01-01,0,2,NaN,dobre warunki atmosferyczne,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,

In [26]:
dane['data_zdarzenia'] = pd.to_datetime(dane['data_zdarzenia'])

In [27]:
#grupuję nasz data frame wg daty i obliczam sumę wypadków przypadających na konkretną datę

new_dane = (dane
    .groupby(['data_zdarzenia'])
    .agg({'liczba_wypadkow':'sum'})
    .sort_values(by='liczba_wypadkow', ascending=False)
    )

In [28]:
#łączę nasze dane z danymi pogodowymi 

new = pd.merge(result, new_dane, left_on='date_time', right_on='data_zdarzenia')

In [29]:
new_corr = new.corr()
new_corr

,maxtempC,mintempC,tempC,totalSnow_cm,cloudcover,humidity,visibility,precipMM,pressure,moon_illumination,hour,liczba_wypadkow
maxtempC,1.000000,0.954491,0.925122,-0.222436,-0.415486,-0.232262,0.166813,0.011041,-0.115904,-0.036986,NaN,0.266903
mintempC,0.954491,1.000000,0.971442,-0.198508,-0.295234,-0.202290,0.150077,0.036630,-0.180109,-0.050610,NaN,0.265516
tempC,0.925122,0.971442,1.000000,-0.200573,-0.271456,-0.216670,0.151717,0.051783,-0.210894,-0.051738,NaN,0.262410
totalSnow_cm,-0.222436,-0.198508,-0.200573,1.000000,0.182493,0.092976,-0.153754,0.118729,-0.077042,-0.053670,NaN,-0.055367
cloudcover,-0.415486,-0.295234,-0.271456,0.182493,1.000000,0.397329,-0.300708,0.172509,-0.207387,0.015129,NaN,-0.118941
humidity,-0.232262,-0.202290,-0.216670,0.092976,0.397329,1.000000,-0.381542,0.094999,-0.091835,0.017603,NaN,0.004688
visibility,0.166813,0.150077,0.151717,-0.153754,-0.300708,-0.381542,1.000000,-0.121333,0.046627,0.031108,NaN,0.027790
precipMM,0.011041,0.036630,0.051783,0.118729,0.172509,0.094999,-0.121333,1.000000,-0.134495,-0.030318,NaN,-0.033566
pressure,-0.115904,-0.180109,-0.210894,-0.077042,-0.207387,-0.091835,0.046627,-0.134495,1.000000,0.041442,NaN,-0.058860
moon_illumination,-0.036986,-0.050610,-0.051738,-0.053670,0.015129,0.017603,0.031108,-0.030318,0.041442,1.000000,NaN,-0.020491


In [30]:
new.sample(5)

# maxtempC: maksymalna temperatura w ciągu dnia w stopniach Celsjusza
# mintempC: minimalna temperatura w ciągu dnia w stopniach Celsjusza
# tempC: temperatura w stopniach Celsjusza
# visibility: widoczność w kilometrach
# moon_illumination: oświetlenie księżyca w % (jaka część księżyca jest widoczna, 100% - księżyc w pełni)

,date_time,maxtempC,mintempC,tempC,totalSnow_cm,cloudcover,humidity,visibility,precipMM,pressure,moon_illumination,location,hour,liczba_wypadkow
978336,2018-08-20,31,21,22,0.0,30,60,10,0.0,1018,59,wreczyca_wielka,0,8
1246385,2019-05-11,20,8,10,0.0,80,91,10,0.6,1013,46,siedlec_duzy,0,8
623597,2017-09-05,13,12,13,0.0,76,72,10,0.0,1016,100,skrzydlow,0,15
1152901,2019-02-08,5,2,0,0.0,99,73,10,0.1,1015,22,rzeki_male,0,10
634916,2017-09-16,13,10,10,0.0,25,77,10,0.0,1015,16,wojkowice,0,10


<span style=color:blue> Liczba wypadków a tempeartura

In [31]:
(new
 .groupby(['tempC'])
 .agg({'liczba_wypadkow':'sum'})
 .sort_values(by='liczba_wypadkow', ascending=False)
 )


,liczba_wypadkow
tempC,
13,736686
14,717994
15,682976
12,645476
16,637037
...,...
41,55
42,26
-32,20


In [32]:
(new[new['tempC'] > 0]
     .groupby(['tempC'])
     .agg({'liczba_wypadkow':'sum'})
     .sort_values(by='liczba_wypadkow', ascending=False))

,liczba_wypadkow
tempC,
13,736686
14,717994
15,682976
12,645476
16,637037
10,629939
11,629111
9,613122
8,577497


In [33]:
#liczba wypadków w dni z temperaturą powyżej 0

print('Liczba wypadków w dni z temperaturą powyżej zera', new[new['tempC'] > 0]['liczba_wypadkow'].sum())
print('Liczba wszystkich wypadków w Katowicach', new['liczba_wypadkow'].sum())
print('Liczba wypadków w dni z temperaturą niższą bądź równą zero', new[new['tempC'] <= 0]['liczba_wypadkow'].sum())

Liczba wypadków w dni z temperaturą powyżej zera 11514336
Liczba wszystkich wypadków w Katowicach 13612368
Liczba wypadków w dni z temperaturą niższą bądź równą zero 2098032


In [34]:
#dane dotyczące dni z temperaturą powyżej 0

new[new['tempC'] > 0].count()

date_time            1195748
maxtempC             1195748
mintempC             1195748
tempC                1195748
totalSnow_cm         1195748
cloudcover           1195748
humidity             1195748
visibility           1195748
precipMM             1195748
pressure             1195748
moon_illumination    1195748
location             1195748
hour                 1195748
liczba_wypadkow      1195748
dtype: int64

In [36]:
#dane dotyczące dni z temperaturą niższą bądź równą zero

new[new['tempC'] <=0].count()

date_time            288628
maxtempC             288628
mintempC             288628
tempC                288628
totalSnow_cm         288628
cloudcover           288628
humidity             288628
visibility           288628
precipMM             288628
pressure             288628
moon_illumination    288628
location             288628
hour                 288628
liczba_wypadkow      288628
dtype: int64

In [37]:
new_data = dane.groupby(['miejscowosc','data_zdarzenia'])
dane.columns
dane.isnull().sum().any()

True